In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pprint, json
import requests
import datetime

PLUTO_URL="http://api.pluto.tv/v2/channels?start=2022-02-20%2023%3A00%3A00.000%2B0100&stop=2022-02-22%2023%3A00%3A00.000%2B0100"
pluto=requests.get(PLUTO_URL)

In [ ]:
j=pluto.json()

In [ ]:
slugs=[]
for e in j:
    slugs.append(e['slug'])
slugs=sorted(list(set(slugs)))

In [ ]:
slugs

In [ ]:
my_channel_slugs=['pluto-tv-sci-fi-de','pluto-tv-sci-fi-de','pluto-tv-star-trek-de']
my_channel_slugs="*"
my_channel_slugs
my_series_slugs=['star-trek-discovery-de', 'star-trek-discovery-ptv2','star-trek-enterprise-de']

In [ ]:
i=0
all=[]
for e in j:
    if my_channel_slugs =="*" or e['slug'] in my_channel_slugs:
        FORMATTER="(%s) %s = %d timelines"
        timelines = e.get('timelines',[])
        print (FORMATTER % (e['slug'], e['name'], len(timelines)))
        for tl in timelines:
            #pprint.pprint(tl)
            ep=tl['episode']
            FORMATTER = "s%02de%02d"
            season = ep.get('season',0)
            exxsxx=FORMATTER % (season, ep['number'])
            start = datetime.datetime.strptime(tl['start'][:19],"%Y-%m-%dT%H:%M:%S")
            stop = datetime.datetime.strptime(tl['stop'][:19],"%Y-%m-%dT%H:%M:%S")
            minutes = int((stop-start).total_seconds() / 60.0)
            start_print = start.strftime("%H:%M %b %d")
            series_name = ep['series']['name']
            series_slug = ep['series']['slug']
            if "Star" not in series_name:
                break
            FORMATTER = "%s %s '%s' %s mins '%s' [%s] %s"
            value=FORMATTER % (exxsxx, start_print, ep['name'], minutes,
                               series_name, series_slug, e['slug'])
            key=e['slug']+"_%s"%start
            print(key)
            print(value)
            all.append((key,value))
            i+=1
        #break

        
print("%s elements" % i)
result = sorted(all, key=lambda x:x[0])
print("\n".join(list(zip(*result))[1]))

zip(*result)

In [ ]:
list(zip(*result))[1]

In [ ]:
def timelines_in_channel(ch, print_channel_name=True):
    timelines = ch.get('timelines',[])
    if print_channel_name:
        FORMATTER="(%s) %s = %d timelines"        
        print (FORMATTER % (ch['slug'], ch['name'], len(timelines)))
    return timelines

def exxsxx(ep):
    FORMATTER = "s%02de%02d"
    season = ep.get('season',0)
    return FORMATTER % (season, ep['number'])

def time_info(tl):
    start = datetime.datetime.strptime(tl['start'][:19],"%Y-%m-%dT%H:%M:%S")
    stop = datetime.datetime.strptime(tl['stop'][:19],"%Y-%m-%dT%H:%M:%S")
    minutes = int((stop-start).total_seconds() / 60.0)
    start_print = start.strftime("%H:%M %b %d")
    return start, start_print, minutes

def series_info(ep):
    series_name = ep['series']['name']
    series_slug = ep['series']['slug']
    return series_name, series_slug


def iterate_pluto(j, find_in_series_name="Star", my_channel_slugs="*", print_channel_name=True, print_each_hit=True):
    hits, ch_slugs, series_slugs = [],[],[]
    for ch in j:
        if my_channel_slugs =="*" or ch['slug'] in my_channel_slugs:
            timelines = timelines_in_channel(ch, print_channel_name)
            for tl in timelines:
                start, start_print, minutes = time_info(tl)
                ep=tl['episode']
                series_name, series_slug = series_info(ep)
                
                if find_in_series_name and (find_in_series_name not in series_name):
                    break
                
                key=ch['slug']+"_%s"%start
                # print(key)
                
                FORMATTER = "%s %s '%s' %s mins '%s' [%s] %s"
                value=FORMATTER % (exxsxx(ep), start_print, ep['name'], minutes,
                                   series_name, series_slug, ch['slug'])
                if print_each_hit:
                    print(value)
                hits.append((key,value))
                ch_slugs.append(ch['slug'])
                series_slugs.append(series_slug)
            #break

    print("\n#################################\n%s elements.\n" % len(hits))
    result = sorted(hits, key=lambda x:x[0])
    return result, sorted(list(set(ch_slugs))), sorted(list(set(series_slugs)))

result, channels, series = iterate_pluto(j, print_channel_name=False)
print("\n".join(list(zip(*result))[1]))
print ("Series: %s\nChannels: %s" % (series, channels))

In [ ]:
# careful 2750 answers:
result, channels, series = iterate_pluto(j, find_in_series_name=False, print_each_hit=False)
print("\n".join(list(zip(*result))[1]))
print ("\n#################\nSeries: %s\n################\nChannels: %s" % (series, channels))

In [ ]:
# Everything with "Discovery":
result, channels, series = iterate_pluto(j, find_in_series_name="Discovery", print_each_hit=False)
print("\n".join(list(zip(*result))[1]))
print ("\n#################\nSeries: %s\n################\nChannels: %s" % (series, channels))

In [12]:
# Everything with "Star":
result, channels, series = iterate_pluto(j, find_in_series_name="Star", print_each_hit=False, print_channel_name=False)
print("\n".join(list(zip(*result))[1]))
print ("#################\nSeries: %s\n################\nChannels: %s" % (series, channels))


#################################
16 elements.

s00e00 22:00 Feb 20 'Pluto TV Star Trek' 43 mins 'Pluto TV Star Trek' [no-info-available] pluto-tv-star-trek-de
s04e01 22:43 Feb 20 'Kobayashi Maru' 63 mins 'Star Trek: Discovery' [star-trek-discovery-de] pluto-tv-star-trek-de
s04e02 23:46 Feb 20 'Anomalie' 60 mins 'Star Trek: Discovery' [star-trek-discovery-de] pluto-tv-star-trek-de
s04e08 00:46 Feb 21 'Alles oder nichts' 61 mins 'Star Trek: Discovery' [star-trek-discovery-de] pluto-tv-star-trek-de
s04e09 01:47 Feb 21 'Rubikon' 51 mins 'Star Trek: Discovery' [star-trek-discovery-de] pluto-tv-star-trek-de
s04e01 02:38 Feb 21 'Kobayashi Maru' 63 mins 'Star Trek: Discovery' [star-trek-discovery-de] pluto-tv-star-trek-de
s04e02 03:41 Feb 21 'Anomalie' 60 mins 'Star Trek: Discovery' [star-trek-discovery-de] pluto-tv-star-trek-de
s04e09 04:41 Feb 21 'Kir'Shara' 47 mins 'Star Trek: Enterprise' [star-trek-enterprise-de] pluto-tv-star-trek-de
s04e10 05:28 Feb 21 'Daedalus' 47 mins 'Star Trek: En